In [8]:
%load_ext autoreload
%autoreload 2

folder = '/home/olga/data_sm/kmer-hashing/classify_coding_vs_noncoding/'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
cd $folder

/mnt/data_sm/olga/kmer-hashing/classify_coding_vs_noncoding


In [10]:
ls -lha

total 239G
drwxrwxr-x 2 olga olga 4.0K Oct 11 22:41 ./
drwxrwxr-x 5 olga olga 4.0K Sep 14 08:39 ../
-rw-rw-r-- 1 olga olga  21M Sep 14 06:46 Homo_sapiens.GRCh38.cds.all.fa.gz
-rw-rw-r-- 1 olga olga  16M Sep 14 06:46 Homo_sapiens.GRCh38.ncrna.fa.gz
-rw-rw-r-- 1 olga olga  14M Sep 14 06:46 Homo_sapiens.GRCh38.pep.all.fa.gz
-rw-rw-r-- 1 olga olga  15G Sep 14 07:11 human_cds_k15.nodegraph
-rw-rw-r-- 1 olga olga   65 Sep 14 07:11 human_cds_k15.nodegraph.info
-rw-rw-r-- 1 olga olga  15G Sep 14 07:22 human_cds_k17.nodegraph
-rw-rw-r-- 1 olga olga   65 Sep 14 07:22 human_cds_k17.nodegraph.info
-rw-rw-r-- 1 olga olga  15G Sep 14 07:30 human_cds_k19.nodegraph
-rw-rw-r-- 1 olga olga   65 Sep 14 07:30 human_cds_k19.nodegraph.info
-rw-rw-r-- 1 olga olga  15G Sep 14 07:39 human_cds_k21.nodegraph
-rw-rw-r-- 1 olga olga   65 Sep 14 07:39 human_cds_k21.nodegraph.info
-rw-rw-r-- 1 olga olga  15G Sep 14 07:47 human_cds_k23.nodegraph
-rw-rw-r-- 1 olga olga   65 Sep 14 07:47 human_cds_k23.nodegraph.info
-r

In [18]:
import gzip
from io import StringIO
from pprint import pprint

from khmer import Nodegraph
import matplotlib.pyplot as plt
import screed
from sourmash._minhash import hash_murmur
from khmer.khmer_args import calculate_graphsize
import pandas as pd
from sourmash.logging import notify
import seaborn as sns
from sklearn import metrics
from tqdm import tqdm
from Bio.Seq import Seq
from Bio import SeqIO

# make all warnings error to get a stack trace
import warnings
warnings.filterwarnings("default")

pd.options.display.max_colwidth = 100

from khtools.compare_kmer_content import kmerize, hpize, dayhoffize
from khtools.partition_reads import score_reads, make_peptide_bloom_filter, six_frame_translation_no_stops, write_fasta, encode_peptide

In [15]:
%%time
peptide_ksize = 7

coding_peptides_bf = make_peptide_bloom_filter("SRR306838_GSM752691_hsa_br_F_1_trimmed_subsampled.coding_peptides.fasta",
                                               peptide_ksize)


CPU times: user 264 ms, sys: 1.4 s, total: 1.67 s
Wall time: 1.66 s


/home/olga/code/kmer-hashing/kh-tools/khtools/partition_reads.py:48: ResourceWarning: unclosed file <_io.BufferedReader name='SRR306838_GSM752691_hsa_br_F_1_trimmed_subsampled.coding_peptides.fasta'>
  for record in screed.open(peptide_fasta):


In [29]:
coding_peptides_bf.n_unique_kmers()

3818

In [39]:
molecule = 'protein'
n = 0

with open("Homo_sapiens.GRCh38.pep.subset.fa", "w") as f:
    with screed.open("Homo_sapiens.GRCh38.pep.all.fa.gz") as records:
        for record in records:
            if '*' in record['sequence']:
                continue
            sequence = encode_peptide(record['sequence'], molecule)
            kmers = kmerize(sequence, peptide_ksize)
            hashes = [hash_murmur(kmer) for kmer in kmers]
            hashes
            if any(coding_peptides_bf.get(h) for h in hashes):
                n += 1
                write_fasta(f, record['name'], record['sequence'])
print(f"N proteins added: {n}")

/home/olga/miniconda3/envs/kmer-hashing/lib/python3.6/site-packages/screed/openscreed.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='Homo_sapiens.GRCh38.pep.all.fa.gz'>
  self.iter_fn = self.open_reader(filename, *args, **kwargs)


N proteins added: 2658


In [22]:
ll 

total 250112113
-rw-rw-r-- 1 olga    21628188 Sep 14 06:46 Homo_sapiens.GRCh38.cds.all.fa.gz
-rw-rw-r-- 1 olga    16335856 Sep 14 06:46 Homo_sapiens.GRCh38.ncrna.fa.gz
-rw-rw-r-- 1 olga    14155330 Sep 14 06:46 Homo_sapiens.GRCh38.pep.all.fa.gz
-rw-rw-r-- 1 olga     2632854 Oct 13 16:57 Homo_sapiens.GRCh38.pep.subset.fa
-rw-rw-r-- 1 olga 16000000036 Sep 14 07:11 human_cds_k15.nodegraph
-rw-rw-r-- 1 olga          65 Sep 14 07:11 human_cds_k15.nodegraph.info
-rw-rw-r-- 1 olga 16000000036 Sep 14 07:22 human_cds_k17.nodegraph
-rw-rw-r-- 1 olga          65 Sep 14 07:22 human_cds_k17.nodegraph.info
-rw-rw-r-- 1 olga 16000000036 Sep 14 07:30 human_cds_k19.nodegraph
-rw-rw-r-- 1 olga          65 Sep 14 07:30 human_cds_k19.nodegraph.info
-rw-rw-r-- 1 olga 16000000036 Sep 14 07:39 human_cds_k21.nodegraph
-rw-rw-r-- 1 olga          65 Sep 14 07:39 human_cds_k21.nodegraph.info
-rw-rw-r-- 1 olga 16000000036 Sep 14 07:47 human_cds_k23.nodegraph
-rw-rw-r-- 1 olga          65 Sep 14 07:47 human_cds_k2

In [41]:
! gzip -c Homo_sapiens.GRCh38.pep.subset.fa > Homo_sapiens.GRCh38.pep.subset.fa.gz

In [42]:
ls -lha Homo_sapiens.GRCh38.pep*

-rw-rw-r-- 1 olga olga  14M Sep 14 06:46 Homo_sapiens.GRCh38.pep.all.fa.gz
-rw-rw-r-- 1 olga olga 2.6M Oct 13 17:26 Homo_sapiens.GRCh38.pep.subset.fa
-rw-rw-r-- 1 olga olga 489K Oct 13 17:26 Homo_sapiens.GRCh38.pep.subset.fa.gz


In [43]:
! head Homo_sapiens.GRCh38.pep.subset.fa

>ENSP00000460880.1 pep chromosome:GRCh38:CHR_HG142_HG150_NOVEL_TEST:56318307:56319245:1 gene:ENSG00000262755.1 transcript:ENST00000573400.1 gene_biotype:polymorphic_pseudogene transcript_biotype:polymorphic_pseudogene gene_symbol:OR8K3 description:olfactory receptor family 8 subfamily K member 3 (gene/pseudogene) [Source:HGNC Symbol;Acc:HGNC:15313]
MEQHNLTTVNEFILTGITDIAELQAPLFALFLMIYVISVMGNLGMIVLTKLDSRLQTPMYFFLRHLAFMDLGYSTTVGPKMLVNFVVDKNIISYYFCATQLAFFLVFIGSELFILSAMSYDLYVAICNPLLYTVIMSRRVCQVLVAIPYLYCTFISLLVTIKIFTLSFCGYNVISHFYCDSLPLLPLLCSNTHEIELIILIFAAIDLISSLLIVLLSYLLILVAILRMNSAGRQKAFSTCGAHLTVVIVFYGTLLFMYVQPKSSHSFDTDKVASIFYTLVIPMLNPLIYSLRNKDVKYALRRTWNNLCNIFV
>ENSP00000354499.2 pep chromosome:GRCh38:MT:5904:7445:1 gene:ENSG00000198804.2 transcript:ENST00000361624.2 gene_biotype:protein_coding transcript_biotype:protein_coding gene_symbol:MT-CO1 description:mitochondrially encoded cytochrome c oxidase I [Source:HGNC Symbol;Acc:HGNC:7419]
MFADRWLFSTNHKDIGTLYLLFGAWAGVLGTALSLLIRAELGQPGNLLGNDHI

In [44]:
pwd

'/mnt/data_sm/olga/kmer-hashing/classify_coding_vs_noncoding'

In [45]:
%%time

peptide_subset_bf = make_peptide_bloom_filter("Homo_sapiens.GRCh38.pep.subset.fa.gz", peptide_ksize)

/home/olga/miniconda3/envs/kmer-hashing/lib/python3.6/site-packages/screed/openscreed.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='Homo_sapiens.GRCh38.pep.subset.fa.gz'>
  self.iter_fn = self.open_reader(filename, *args, **kwargs)


CPU times: user 1.99 s, sys: 1.78 s, total: 3.77 s
Wall time: 3.77 s


In [46]:
peptide_subset_bf.n_unique_kmers()

509335

In [48]:
molecules = 'protein', 'dayhoff', 'hydrophobic-polar'
ksizes = 7, 8

for molecule in molecules:
    for ksize in ksizes:
        print(f'molecule: {molecule}, ksize: {ksize}')
        bf = make_peptide_bloom_filter("Homo_sapiens.GRCh38.pep.subset.fa.gz", ksize, molecule, tablesize=1e6)
        print(bf.n_unique_kmers())

molecule: protein, ksize: 7


/home/olga/miniconda3/envs/kmer-hashing/lib/python3.6/site-packages/screed/openscreed.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='Homo_sapiens.GRCh38.pep.subset.fa.gz'>
  self.iter_fn = self.open_reader(filename, *args, **kwargs)


506352
molecule: protein, ksize: 8
512787
molecule: dayhoff, ksize: 7
99863
molecule: dayhoff, ksize: 8
223991
molecule: hydrophobic-polar, ksize: 7
170
molecule: hydrophobic-polar, ksize: 8
317


In [37]:
bf == peptide_subset_bf

False